In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# 데이터 로드
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import cross_val_score

# 데이터 로드
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# 데이터 결합
all_data = pd.concat((train.loc[:, 'MSSubClass':'SaleCondition'],
                      test.loc[:, 'MSSubClass':'SaleCondition']), ignore_index=True)

# Quantile Transformer 적용
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
all_data[numeric_feats] = quantile_transformer.fit_transform(all_data[numeric_feats])

# 범주형 변수를 더미 변수로 변환
all_data = pd.get_dummies(all_data)

# 결측값을 각 열의 평균 값으로 채우기
all_data = all_data.fillna(all_data.mean())

# 학습 데이터와 테스트 데이터 분리
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]

# 타겟 값 설정 및 로그 변환
y = train["SalePrice"]
y_log = np.log1p(y)

# 결과 확인
print(X_train.shape, X_test.shape, y.shape)

(1460, 287) (1459, 287) (1460,)


In [36]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, LassoCV, LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

In [37]:
# Lasso Regression
lasso = Lasso(alpha=0.1, max_iter=10000)
lasso_scores = cross_val_score(lasso, X_train, y_log, cv=5, scoring='neg_mean_squared_error')
lasso_rmse = (-lasso_scores.mean()) ** 0.5
print(f'Lasso RMSE: {lasso_rmse}')

# Ridge Regression
ridge = Ridge(alpha=0.1, max_iter=10000)
ridge_scores = cross_val_score(ridge, X_train, y_log, cv=5, scoring='neg_mean_squared_error')
ridge_rmse = (-ridge_scores.mean()) ** 0.5
print(f'Ridge RMSE: {ridge_rmse}')


Lasso RMSE: 0.20562516772559178
Ridge RMSE: 0.1418789147966438


In [38]:
# Lasso 하이퍼파라미터 튜닝
lasso_params = {'alpha': [1, 0.1, 0.001, 0.0005]}
lasso_grid = GridSearchCV(Lasso(), lasso_params, cv=5, scoring='neg_mean_squared_error')
lasso_grid.fit(X_train, y_log)
print(f'Best Lasso Alpha: {lasso_grid.best_params_}')
print(f'Best Lasso RMSE: {(-lasso_grid.best_score_) ** 0.5}')

# Ridge 하이퍼파라미터 튜닝
ridge_params = {'alpha': [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]}
ridge_grid = GridSearchCV(Ridge(), ridge_params, cv=5, scoring='neg_mean_squared_error')
ridge_grid.fit(X_train, y_log)
print(f'Best Ridge Alpha: {ridge_grid.best_params_}')
print(f'Best Ridge RMSE: {(-ridge_grid.best_score_) ** 0.5}')

Best Lasso Alpha: {'alpha': 0.0005}
Best Lasso RMSE: 0.12737599318977683
Best Ridge Alpha: {'alpha': 15}
Best Ridge RMSE: 0.13096738520223392


In [42]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(n_estimators=400, max_depth=2, learning_rate=0.1)
xgb_scores = cross_val_score(xgb_model, X_train, y_log, cv=5, scoring='neg_mean_squared_error')
xgb_rmse = (-xgb_scores.mean()) ** 0.5
print(f'XGBoost RMSE: {xgb_rmse}')
#튜닝한번했음

XGBoost RMSE: 0.12401890459263881


In [44]:
random_forest = RandomForestRegressor(n_estimators=100, max_depth=5)

In [47]:
# Base models
estimators = [
    ('lasso', Lasso(alpha=lasso_grid.best_params_['alpha'])),
    ('ridge', Ridge(alpha=ridge_grid.best_params_['alpha'])),
    ('random_forest', random_forest),
    ('xgb', XGBRegressor(n_estimators=400, max_depth=2, learning_rate=0.1))
]

# Stacking model
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)

# Train the stacking model
stacking_model.fit(X_train, y_log)


# 교차 검증을 통한 성능 평가
stacking_scores = cross_val_score(stacking_model, X_train, y_log, cv=5, scoring='neg_mean_squared_error')
stacking_rmse = (-stacking_scores.mean()) ** 0.5
print(f'Stacking RMSE: {stacking_rmse}')

# Test 데이터에 대한 예측
y_pred = stacking_model.predict(X_test)

# 제출 파일 생성
submission = pd.DataFrame({'Id': test['Id'], 'SalePrice': np.expm1(y_pred)})
submission.to_csv('submission.csv', index=False)

Stacking RMSE: 0.11932535363714404


## Ridge,Lasso,XGBRegressor 모두 한번씩 구해보고 하이퍼 파라미터 튜닝까지
## Lasso Ridge 둘다 선형이니까 둘중에 하나만 택하고
## random forest 까지 추가해준다음에 stacking